In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd
from ipywidgets import interact
import seaborn as sns
#import ptitprince as pt
import re #I import this library 'cause I need to deal with regular expressions
from sklearn.decomposition import PCA
from mlxtend.plotting import plot_decision_regions
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn import tree
#from yellowbrick.cluster import KElbowVisualizer
from sklearn import metrics
sns.set_style('whitegrid')
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from category_encoders import TargetEncoder
from sklearn.preprocessing import PolynomialFeatures

In [6]:
import stanza
stanza.download('es')


2024-04-09 12:05:56 INFO: Downloaded file to C:\Users\jlsan\stanza_resources\resources.json
2024-04-09 12:05:56 INFO: Downloading default packages for language: es (Spanish) ...


2024-04-09 12:06:12 INFO: Downloaded file to C:\Users\jlsan\stanza_resources\es\default.zip
2024-04-09 12:06:15 INFO: Finished downloading models and saved to C:\Users\jlsan\stanza_resources


In [7]:
main_file="C:/Users/jlsan/OneDrive/JEP_LARGE_DATASET/DATA.csv"
data=pd.read_csv(main_file)
pd.set_option('display.max_colwidth', None)

In [9]:
data_df=pd.DataFrame(data)
data_df.head(1)

INDEX  \
0  A001-00   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [10]:
nlp = stanza.Pipeline('es')


2024-04-09 12:09:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-09 12:09:25 INFO: Downloaded file to C:\Users\jlsan\stanza_resources\resources.json
2024-04-09 12:09:27 INFO: Loading these models for language: es (Spanish):
| Processor    | Package         |
----------------------------------
| tokenize     | ancora          |
| mwt          | ancora          |
| pos          | ancora_charlm   |
| lemma        | ancora_nocharlm |
| constituency | combined_charlm |
| depparse     | ancora_charlm   |
| sentiment    | tass2020_charlm |
| ner          | conll02         |

2024-04-09 12:09:27 INFO: Using device: cpu
2024-04-09 12:09:27 INFO: Loading: tokenize
2024-04-09 12:09:28 INFO: Loading: mwt
2024-04-09 12:09:28 INFO: Loading: pos
2024-04-09 12:09:29 INFO: Loading: lemma
2024-04-09 12:09:29 INFO: Loading: constituency
2024-04-09 12:09:29 INFO: Loading: depparse
2024-04-09 12:09:29 INFO: Loading: sentiment
2024-04-09 12:09:30 INFO: Loading: ner
2024-04-09 12:09:30 INFO: Done loading processors!


In [15]:
len(data_df)

35737

In [16]:
def ent_extractor(doc):
    ent_nlp = nlp(doc)
    ext_ent = [(ent.text, ent.type) for ent in ent_nlp.ents]
    return ext_ent
#data_df['entidades'] = data_df['TEXT'].apply(ent_extractor)
primer_registro_entidades = ent_extractor(data_df['TEXT'].iloc[0])


In [17]:
primer_registro_entidades

[('Lorenza Díaz Padilla', 'PER'),
 ('Hospital San Jerónimo', 'ORG'),
 ('Montería', 'LOC'),
 ('Magistrado Sustanciador', 'PER'),
 ('Dr', 'PER'),
 ('JOSÉ GREGORIO HERNÁNDEZ', 'PER'),
 ('Santafé de Bogotá D.C.', 'LOC'),
 ('I. ANTECEDENTES', 'MISC'),
 ('San Jerónimo', 'LOC'),
 ('Montería', 'LOC'),
 ('Hospital', 'ORG'),
 ('Empresa Social del Estado', 'ORG'),
 ('Hospital San Jerónimo de Montería', 'ORG'),
 ('DECISION JUDICIAL', 'MISC'),
 ('Tribunal Superior del Distrito Judicial de Montería -Sala Civil Familia-',
  'ORG'),
 ('Lorenza Díaz Padilla', 'PER'),
 ('Sala Quinta de Revisión de la Corte Constitucional', 'ORG'),
 ('Constitución', 'MISC'),
 ('Tribunal Superior del Distrito Judicial de Montería', 'ORG'),
 ('ABSTENERSE', 'ORG'),
 ('Tribunal Superior de Montería', 'ORG'),
 ('Secretaría General de esta Corporación', 'ORG'),
 ('Sala Civil', 'ORG'),
 ('Familia del Tribunal Superior de Montería', 'ORG'),
 ('Sala', 'ORG'),
 ('Gaceta de la Corte Constitucional', 'ORG'),
 ('JOSE GREGORIO HERNAND

In [11]:
doc = nlp("Albert Einstein was born in Ulm, Germany.")
print(doc.ents)


[{
  "text": "Albert Einstein",
  "type": "PER",
  "start_char": 0,
  "end_char": 15
}, {
  "text": "Ulm",
  "type": "LOC",
  "start_char": 28,
  "end_char": 31
}, {
  "text": "Germany",
  "type": "LOC",
  "start_char": 33,
  "end_char": 40
}]


In [19]:
segundo_registro_entidades = ent_extractor(data_df['TEXT'].iloc[1])

In [20]:
segundo_registro_entidades

[('Corte', 'ORG'),
 ('INHIBICION DE LA CORTE CONSTITUCIONAL-Resolución', 'MISC'),
 ('Sentencia C-1063', 'MISC'),
 ('Leoncio Ceballos', 'PER'),
 ('Magistrada Ponente', 'PER'),
 ('Dra', 'PER'),
 ('CRISTINA PARDO', 'PER'),
 ('Bogotá', 'LOC'),
 ('Corte Constitucional de la República de Colombia', 'ORG'),
 ('Fabio Morón Díaz', 'PER'),
 ('Eduardo Montealegre Lynett', 'PER'),
 ('Alfredo Beltrán Sierra', 'PER'),
 ('Martha Sáchica de Moncaleano', 'PER'),
 ('Carlos Gaviria Díaz', 'PER'),
 ('José Gregorio Hernández Galindo', 'PER'),
 ('Alejandro Martínez Caballero', 'PER'),
 ('Cristina Pardo Schlesinger', 'PER'),
 ('Álvaro Tafur Gálvi', 'PER'),
 ('AUTO', 'MISC'),
 ('Leoncio Ceballos Valencia', 'PER'),
 ('Sentencia C-1063', 'MISC'),
 ('Corte Constitucional', 'ORG'),
 ('articulo 1', 'MISC'),
 ('Decreto 085', 'MISC'),
 ('L', 'LOC'),
 ('artículo 49', 'MISC'),
 ('Decreto 2067', 'MISC'),
 ('Corte', 'ORG'),
 ('Corporación', 'ORG'),
 ('Corte Constitucional', 'ORG'),
 ('Corte Constitucional', 'ORG'),
 ('C

In [25]:
terminos_derechos_humanos = [
    'derechos humanos', 'libertad de expresión', 'igualdad de género', 'discriminación', 
    'trabajo infantil', 'asilo', 'refugiados', 'debido proceso', 'libertad de asociación', 
    'libre pensamiento', 'libertad de culto', 'seguridad personal', 'desaparecido', 
    'intimidad', 'libertad de opinión', 'no discriminación', 'libertad sindical'
]


In [53]:
def sentence_finder(textos):  
    mark_sentences = []
    for idx,texto in textos:  
        doc = nlp(texto)
        for sentencia in doc.sentences:  
            text_sentence = sentencia.text.lower() 
            if any(term in text_sentence for term in terminos_derechos_humanos):  
                mark_sentences.append((idx,sentencia.text))  
    return mark_sentences


In [54]:
txt_id=list(zip(data_df.index,data_df['TEXT']))

In [55]:
size = 5
all_results = []

for i in range(0, 10, size):
    end = i + size
    batch = txt_id[i:end]
    processed_batch = sentence_finder(batch)
    all_results.extend(processed_batch)


size=5
results=[]

for i in range(0,10,size):
    end=i+size
    batch=data_df['TEXT'].iloc[i:end].tolist()
    processed_batch=sentence_finder(batch)
    results.extend(processed_batch)


    
    

In [56]:
df_oraciones_encontradas = pd.DataFrame(all_results, columns=['ID_Documento', 'Oración'])

In [57]:
df_oraciones_encontradas

,ID_Documento,Oración
0,0,DEBIDO PROCESO DE TUTELA-Notificación iniciación de la acción
1,0,"Si bien el trámite propio de la acción de tutela es breve, sumario e informal, toda vez que busca la protección inmediata de un derecho fundamental cuando quiera que éste se vea amenazado o vulnerado por la acción u omisión de una autoridad pública o de un particular en los casos expresamente señalados, no puede llevarse a cabo el proceso sin el conocimiento de la autoridad o del particular contra quien se impetró, pues de lo contrario se le estaría vulnerando en forma flagrante su derecho a la defensa y por ende al debido proceso."
2,0,"Si bien el trámite propio de la acción de tutela es breve, sumario e informal, toda vez que busca la protección inmediata de un derecho fundamental cuando quiera que éste se vea amenazado o vulnerado por la acción u omisión de una autoridad pública o de un particular en los casos expresamente señalados, no puede llevarse a cabo el proceso sin el conocimiento de la autoridad o del particular contra quien se impetró, pues de lo contrario se le estaría vulnerando en forma flagrante su derecho a la defensa y por ende al debido proceso."
3,1,"La Corte recuerda que, en asuntos de constitucionalidad, la nulidad de una sentencia será siempre un asunto extraordinario que pueda producirse en uno de los dos momentos: en el del trámite procesal, o en el de la sentencia misma, y en ambos casos por violación del debido proceso."
4,1,"En efecto, con fundamento en ese artículo, la Corte ha admitido solicitudes de nulidad de procesos de constitucionalidad, no sólo por actuaciones o hechos ocurridos antes de dictarse el fallo sino también por irregularidades en la sentencia misma, siempre y cuando éstas impliquen violación del debido proceso, pues esta Corporación ha precisado que tiene ""el deber de declarar las nulidades que se presenten en cualquier etapa del proceso."
5,1,"""Lo dicho no ha sido obstáculo para que la misma Corte, dando aplicación directa al artículo 29 de la Carta Política, haya admitido que puede darse excepcionalmente en sus sentencias la posibilidad de violaciones del debido proceso, durante el trámite judicial correspondiente o en el momento de dictarse, por lo cual, si ello se prueba y establece de manera contundente, tiene lugar la nulidad del respectivo fallo (Cfr."
6,1,"""Dispone el precepto legal que sólo las irregularidades que impliquen violación del debido proceso podrán servir de base para que la Sala Plena de la Corte anule el proceso."
7,1,"Se trata de situaciones jurídicas especialísimas y excepcionales, que tan sólo pueden provocar la nulidad del proceso cuando los fundamentos expuestos por quien la alega muestran, de manera indudable y cierta, que las reglas procesales aplicables a los procesos constitucionales, que no son otras que las previstas en los decretos 2067 y 2591 de 1991, han sido quebrantadas, con notoria y flagrante vulneración del debido proceso."
8,1,"Toda sentencia desfavorable disgusta y molesta a quien no fue beneficiado por la decisión que contiene, pero de esa molestia y disgusto no puede deducirse irresponsablemente una vulneración del debido proceso por el sólo hecho de que se trata de una providencia definitiva contra la cual no cabe ningún recurso""."
9,1,"Conforme con lo anterior, la Corte recuerda que, en asuntos de constitucionalidad, la nulidad de una sentencia será siempre un asunto extraordinario que pueda producirse en uno de los dos momentos: en el del trámite procesal, o en el de la sentencia misma, y en ambos casos por violación del debido proceso."
